<a href="https://colab.research.google.com/github/deepakawl/supplychain-analytics-teaching/blob/main/getExchangeRates_Frankfurter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1

In [1]:
import requests
from datetime import datetime, timedelta, date
import pandas as pd
from scipy.stats import multivariate_normal

def get_month_end_dates(start_date_str, end_date_str):
    """
    Generate a list of dates representing the last day of each month
    between start_date and end_date. If the end_date falls in the middle
    of a month, that date will be used for the final month.
    """
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d').date()
    end_date   = datetime.strptime(end_date_str, '%Y-%m-%d').date()
    month_end_dates = []
    # Start from the first day of the month of start_date.
    current = start_date.replace(day=1)
    while current <= end_date:
        # Determine the first day of the next month.
        if current.month == 12:
            next_month = date(current.year + 1, 1, 1)
        else:
            next_month = date(current.year, current.month + 1, 1)
        # Last day of the current month is one day before the first day of the next month.
        last_day_of_month = next_month - timedelta(days=1)
        # If the last day of the month is after the specified end_date,
        # then use end_date (if it is in the same month) instead.
        if last_day_of_month > end_date:
            last_day_of_month = end_date
        if last_day_of_month >= start_date:
            month_end_dates.append(last_day_of_month)
        # Move to the first day of the next month.
        current = next_month
    return month_end_dates

def get_exchange_rate_for_date(date_str, base_currency="USD", symbols=None):
    """
    Fetches exchange rates for the given date from the Frankfurter API.

    :param date_str: Date in 'YYYY-MM-DD' format.
    :param base_currency: Base currency (default "USD").
    :param symbols: List of target currencies.
    :return: Dictionary of exchange rates.
    """
    url = f"https://api.frankfurter.app/{date_str}"
    params = {"from": base_currency}
    if symbols:
        params["to"] = ",".join(symbols)
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data.get("rates", {})
    else:
        print(f"Error fetching data for {date_str}: HTTP {response.status_code}")
        return {}

def estimate_statistics(exchange_data):
    """
    Computes the average exchange rate vector and variance-covariance matrix.

    :param exchange_data: dict with keys as date strings and values as rate dictionaries.
    :return: (average_vector, covariance_matrix)
    """
    df = pd.DataFrame.from_dict(exchange_data, orient="index")
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)
    average_vector = df.mean()
    covariance_matrix = df.cov()
    return average_vector, covariance_matrix

# Specify the period and target currencies.
start_date_str = "2015-01-01"
end_date_str   = "2024-12-31"
target_currencies = ["BRL", "EUR", "INR", "JPY", "MXN"]

# Get month-end dates.
month_end_dates = get_month_end_dates(start_date_str, end_date_str)

# Fetch exchange rates for each month-end date.
exchange_data = {}
for d in month_end_dates:
    date_str = d.isoformat()
    print(f"Fetching rates for {date_str}...")
    rates = get_exchange_rate_for_date(date_str, base_currency="USD", symbols=target_currencies)
    exchange_data[date_str] = rates

# Display the collected exchange rates.
print("\nExchange Rates at Month End:")
for date_str, rates in exchange_data.items():
    print(f"{date_str}: {rates}")

# Estimate the average vector and covariance matrix.
avg_vector, cov_matrix = estimate_statistics(exchange_data)

print("\nAverage Exchange Rate Vector:")
print(avg_vector)

print("\nVariance-Covariance Matrix:")
print(cov_matrix)

# Create the multivariate normal distribution based on estimated statistics.
mvn_distribution = multivariate_normal(mean=avg_vector, cov=cov_matrix)

# Draw a sample.
sample = mvn_distribution.rvs(size=1, random_state=1)
print("\nSampled draw:", sample)

Fetching rates for 2015-01-31...
Fetching rates for 2015-02-28...
Fetching rates for 2015-03-31...
Fetching rates for 2015-04-30...
Fetching rates for 2015-05-31...
Fetching rates for 2015-06-30...
Fetching rates for 2015-07-31...
Fetching rates for 2015-08-31...
Fetching rates for 2015-09-30...
Fetching rates for 2015-10-31...
Fetching rates for 2015-11-30...
Fetching rates for 2015-12-31...
Fetching rates for 2016-01-31...
Fetching rates for 2016-02-29...
Fetching rates for 2016-03-31...
Fetching rates for 2016-04-30...
Fetching rates for 2016-05-31...
Fetching rates for 2016-06-30...
Fetching rates for 2016-07-31...
Fetching rates for 2016-08-31...
Fetching rates for 2016-09-30...
Fetching rates for 2016-10-31...
Fetching rates for 2016-11-30...
Fetching rates for 2016-12-31...
Fetching rates for 2017-01-31...
Fetching rates for 2017-02-28...
Fetching rates for 2017-03-31...
Fetching rates for 2017-04-30...
Fetching rates for 2017-05-31...
Fetching rates for 2017-06-30...
Fetching r

# Question 2

In [2]:
#%% Reset and Install Packages
%reset -f
!pip install forex_python
!pip install gurobipy
!pip install tabulate

import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum
from tabulate import tabulate
import datetime as dt
from forex_python.converter import get_rate
from scipy.stats import multivariate_normal

_empty_series = pd.Series(dtype=float)

#%% Raw Data and Parameters
base_yr = 2019  # Base year for exchange rate adjustment

# Demand Data
demand = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'd_h': [7, 15, 5, 7, 3, 18],
    'd_r': [7, 12, 3, 8, 3, 17],
})
demand.set_index('from', inplace=True)

# Capacities
caps = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'cap': [18, 45, 18, 10, 30, 22],
})
caps.set_index('plant', inplace=True)

# Product Costs
pcosts = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'fc_p': [20, 45, 14, 13, 30, 23],
    'fc_h': [5, 13, 3, 4, 6, 5],
    'fc_r': [5, 13, 3, 4, 6, 5],
    'rm_h': [3.6, 3.9, 3.6, 3.9, 3.6, 3.6],
    'pc_h': [5.1, 6.0, 4.5, 6.0, 5.0, 5.0],
    'rm_r': [4.6, 5.0, 4.5, 5.1, 4.6, 4.5],
    'pc_r': [6.6, 7.0, 6.0, 7.0, 6.5, 6.5],
})
pcosts.set_index('plant', inplace=True)

# Transportation Costs
tcosts = pd.DataFrame({
    'from': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'LatinAmerica': [0.20, 0.45, 0.50, 0.50, 0.40, 0.45],
    'Europe':       [0.45, 0.20, 0.35, 0.40, 0.30, 0.30],
    'AsiaWoJapan':  [0.50, 0.35, 0.20, 0.30, 0.50, 0.45],
    'Japan':        [0.50, 0.40, 0.30, 0.10, 0.45, 0.45],
    'Mexico':       [0.40, 0.30, 0.50, 0.45, 0.20, 0.25],
    'U.S.':         [0.45, 0.30, 0.45, 0.45, 0.25, 0.20],
})
tcosts.set_index('from', inplace=True)

# Duties
duties = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'duty': [0.30, 0.03, 0.27, 0.06, 0.35, 0.04],
})
duties.set_index('from', inplace=True)

# Provided historical exchange rates (for base year adjustment)
exrate0 = {
    '2018': [3.88, 4.33, 69.63, 109.91, 19.64, 1],
    '2019': [4.33, 0.92, 71.48, 109.82, 18.65, 1],
    '2020': [5.19, 0.82, 73.66, 103.24, 19.90, 1],
    '2021': [5.26, 0.88, 74.28, 115.59, 20.62, 1],
    '2022': [5.29, 0.93, 82.75, 131.12, 19.48, 1],
    '2023': [4.85, 0.91, 83.04, 140.99, 16.96, 1],
}
exrate0 = pd.DataFrame(exrate0, index=['BRL','EUR','INR','JPY','MXN','USD'])

# Mapping of plants to their local currencies.
plant_currency = {'Brazil': 'BRL', 'Germany': 'EUR', 'India': 'INR',
                  'Japan': 'JPY', 'Mexico': 'MXN', 'U.S.': 'USD'}

# Model dimensions.
n_ctry = range(demand.shape[0])       # 6 locations.
n_lines = range(demand.shape[1] + 1)    # 3 decisions per plant: Plant, HighCal, Relax.

#----- Exchange Rate Distribution (from Q1) -----
# These values were computed in Question 1.
avg_vector = pd.Series([4.389619, 0.895970, 72.852125, 120.345750, 19.028722],
                       index=["BRL", "EUR", "INR", "JPY", "MXN"])
cov_matrix = pd.DataFrame([
    [0.866835, 0.007135, 5.424984, 6.776487, 0.756875],
    [0.007135, 0.001784, 0.129991, 0.427207, -0.007570],
    [5.424984, 0.129991, 47.403537, 83.177478, 2.410975],
    [6.776487, 0.427207, 83.177478, 254.423705, -8.938660],
    [0.756875, -0.007570, 2.410975, -8.938660, 2.970507]
], index=["BRL", "EUR", "INR", "JPY", "MXN"],
   columns=["BRL", "EUR", "INR", "JPY", "MXN"])

mvn_distribution = multivariate_normal(mean=avg_vector, cov=cov_matrix, seed=42)

#----- Modified Cost Function Using Exchange Rate Sampling -----
def calc_total_cost(dec_plant, dec_h, dec_r, base_yr=2019, selected_yr=2023, tariff=0, use_sampling=True):
    x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    if use_sampling:
        sampled_values = mvn_distribution.rvs(size=1)
        sampled_rates = pd.Series(sampled_values[0], index=["BRL", "EUR", "INR", "JPY", "MXN"])
        full_sampled = pd.concat([sampled_rates, pd.Series({'USD': 1.0})])
        reindx = exrate0.loc[:, f'{base_yr}'] / full_sampled
    else:
        reindx = pd.Series(1, index=exrate0.index)

    pcosts_rev = pd.DataFrame(pcosts.values * reindx.values.reshape(-1, 1),
                              columns=pcosts.columns, index=pcosts.index)

    duties_mat = pd.DataFrame(np.zeros((len(pcosts_rev), len(duties))) + (1 + duties['duty'].values),
                              index=pcosts_rev.index, columns=duties.index)
    np.fill_diagonal(duties_mat.values, 1)
    duties_mat.loc['Germany', 'U.S.'] += tariff
    duties_mat.loc['U.S.', 'Europe']  += tariff

    vcosts_h = tcosts.add(pcosts_rev['rm_h'], axis=0).add(pcosts_rev['pc_h'], axis=0) * duties_mat
    vcosts_r = tcosts.add(pcosts_rev['rm_r'], axis=0).add(pcosts_rev['pc_r'], axis=0) * duties_mat

    fc = pcosts_rev[['fc_p','fc_h','fc_r']].values
    vh = (vcosts_h * x_h).values
    vr = (vcosts_r * x_r).values
    total_cost = (sum(0.2 * fc[i,j] for i in n_ctry for j in n_lines) +
                  sum(0.8 * fc[i,j] * x_plant[i,j] for i in n_ctry for j in n_lines) +
                  sum(vh[i,j] for i in n_ctry for j in n_ctry) +
                  sum(vr[i,j] for i in n_ctry for j in n_ctry))
    return total_cost

def calc_excess_cap(dec_plant, dec_h, dec_r):
    x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))
    excess_cap = (x_plant * caps.values).copy()
    excess_cap[:, 0] -= (np.sum(x_h, axis=1) + np.sum(x_r, axis=1))
    excess_cap[:, 1] -= np.sum(x_h, axis=1)
    excess_cap[:, 2] -= np.sum(x_r, axis=1)
    return excess_cap

def calc_unmet_demand(dec_h, dec_r):
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))
    x_h_sum = np.sum(x_h, axis=0)
    x_r_sum = np.sum(x_r, axis=0)
    unmet_demand = np.column_stack((x_h_sum - demand['d_h'].values, x_r_sum - demand['d_r'].values))
    return unmet_demand

#----- User Input -----
while True:
    try:
        selected_yr = int(input("Enter year (as yyyy, e.g. 2023): "))
        if 1995 <= selected_yr <= 2024:
            if selected_yr < 2018 or selected_yr > 2023:
                print("Extracting exchange rate from the web. Please hold.")
            break
        else:
            print("Invalid input. Please enter a year between 1995 and 2024.")
    except ValueError:
        print("Invalid input. Please enter a valid year (yyyy).")

while True:
    try:
        tariff = float(input("Enter tariff (in percent, e.g. 10 for 10%): "))
        if 0 <= tariff <= 1000:
            tariff = tariff / 100
            break
        else:
            print("Invalid input. Please enter a valid number between 0 and 1000.")
    except ValueError:
        print("Invalid input. Please enter a valid number.")

#----- Run 100 Samples for Question 2 -----
results = []

for s in range(100):
    model = Model("NetworkDesign_Sample_" + str(s))
    model.Params.OutputFlag = 0  # Suppress solver output

    dec_plant = {(i, j): model.addVar(vtype=GRB.BINARY, name=f"Dec_plant_{i}_{j}")
                 for i in n_ctry for j in n_lines}
    dec_h = {(i, j): model.addVar(vtype=GRB.INTEGER, lb=0, name=f"Dec_h_{i}_{j}")
             for i in n_ctry for j in n_ctry}
    dec_r = {(i, j): model.addVar(vtype=GRB.INTEGER, lb=0, name=f"Dec_r_{i}_{j}")
             for i in n_ctry for j in n_ctry}

    excess_cap = calc_excess_cap(dec_plant, dec_h, dec_r)
    for i in n_ctry:
        for j in n_lines:
            model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_{i}_{j}")

    unmet_demand = calc_unmet_demand(dec_h, dec_r)
    for i in n_ctry:
        for j in range(2):
            model.addConstr(unmet_demand[i, j] == 0, name=f"Unmet_Demand_{i}_{j}")

    model.update()

    # Set objective; each call draws a new sample.
    obj_val = calc_total_cost(dec_plant, dec_h, dec_r, base_yr, selected_yr, tariff, use_sampling=True)
    model.setObjective(obj_val, GRB.MINIMIZE)
    model.optimize()

    op_plant = pd.DataFrame([[dec_plant[i, j].X for j in n_lines] for i in n_ctry],
                            columns=['Plant','H','R'], index=caps.index)
    op_h = pd.DataFrame([[dec_h[i, j].X for j in n_ctry] for i in n_ctry],
                        columns=tcosts.columns, index=tcosts.index)
    op_r = pd.DataFrame([[dec_r[i, j].X for j in n_ctry] for i in n_ctry],
                        columns=tcosts.columns, index=tcosts.index)

    print(f"\nSample {s}:")
    print("HighCal Flow:")
    print(tabulate(op_h, headers='keys', tablefmt='pretty'))
    print("Relax Flow:")
    print(tabulate(op_r, headers='keys', tablefmt='pretty'))
    print("Strategy (Plant decisions):")
    print(tabulate(op_plant, headers='keys', tablefmt='pretty'))
    print(f"Cost: {round(model.objVal,2)}\n")

    results.append({
        "sample": s,
        "optimal_configuration": op_plant['Plant'].tolist(),
        "total_cost": model.objVal
    })

results_df = pd.DataFrame(results)
results_df.to_csv("Q2_results.csv", index=False)
print("Results for Question 2 saved in Q2_results.csv")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 52.4 MB/s eta 0:00:00
Enter year (as yyyy, e.g. 2023): 2024
Extracting exchange rate from the web. Please hold.
Enter tariff (in percent, e.g. 10 for 10%): 10
Restricted license - for non-production use only - expires 2026-11-23

Sample 0:
HighCal Flow:
+---------+--------------+--------+-------------+-------+--------+------+
|  from   | LatinAmerica | Europe | AsiaWoJapan | Japan | Mexico | U.S. |
+---------+--------------+--------+-------------+-------+--------+------+
| Brazil  |     7.0      |  0.0   |     3.0     |  1.0  |  0.0   | -0.0 |
| Germany |     0.0      |  -0.0  |     2.0     | -0.0  |  -0.0  | -0.0 |
|  India  |     0.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
|  Japan  |     0.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
| Mexico  |     0.0      |  15.0  |     0.0     | -0.0  |  3.0   | 2.0  |
|  U.S.   |     -0.0     |  -0.0  |    -0.0     |  6.0  |  0.0   | 16.0 |
+---------+------------

# Question 3 AND 4

In [4]:
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal

#-----------------------------------------------------------
# 1. Read Q2 results from file (assumed saved as "Q2_results.csv")
#    The file is expected to have columns such as:
#    Brazil_Plant, Brazil_H, Brazil_R, Germany_Plant, Germany_H, Germany_R, etc.
#-----------------------------------------------------------
q2_results = pd.read_csv("Q2_results.csv")
print("Q2 Results (first 5 rows):")
print(q2_results.head())

#-----------------------------------------------------------
# 2. Compute the average (mean) decision value for each decision variable over 100 samples.
#    (For binary decisions the average near 1 means “mostly open” and near 0 means “mostly closed”.)
#-----------------------------------------------------------
avg_decisions = q2_results.mean(numeric_only=True)
print("\nAverage decision values from Q2:")
print(avg_decisions)

# For clarity, let’s assume that for each plant we have three columns:
#   {Plant}_Plant, {Plant}_H, {Plant}_R
# For example, 'Brazil_Plant', 'Germany_Plant', 'India_Plant', 'Japan_Plant', 'Mexico_Plant', 'U.S._Plant'

#-----------------------------------------------------------
# 3. Construct Candidate Network Strategies Based on Q2 Averages
#    Here we propose three candidate strategies.
#
# Candidate Strategy 1: Use a 0.5 threshold—if the average decision >= 0.5, then set that decision = 1, else 0.
# Candidate Strategy 2: Modify Strategy 1 by enforcing a known correlation:
#     For example, if Germany's plant is open, then force Japan's plant decision to be closed.
# Candidate Strategy 3: Manually “fix” the network to open the plants that are “typically” open (Brazil, Germany, Mexico, U.S.)
#     and close those that are rarely open (India and Japan).
#-----------------------------------------------------------

# (a) Candidate Strategy 1: Threshold at 0.5 for each decision
candidate_strategy_1 = (avg_decisions >= 0.5).astype(int)

# (b) Candidate Strategy 2: Based on candidate 1, enforce that if Germany_Plant is open, then force Japan_Plant to 0.
candidate_strategy_2 = candidate_strategy_1.copy()
if candidate_strategy_2.get('Germany_Plant', 0) == 1 and candidate_strategy_2.get('Japan_Plant', 0) == 1:
    candidate_strategy_2['Japan_Plant'] = 0

# (c) Candidate Strategy 3: Manually set: open Brazil, Germany, Mexico, U.S. and close India and Japan.
candidate_strategy_3 = candidate_strategy_1.copy()
for plant in ['Brazil', 'Germany', 'Mexico', 'U.S.']:
    candidate_strategy_3[f"{plant}_Plant"] = 1
for plant in ['India', 'Japan']:
    candidate_strategy_3[f"{plant}_Plant"] = 0

print("\nCandidate Strategy 1 (based on thresholding):")
print(candidate_strategy_1)
print("\nCandidate Strategy 2 (enforcing Germany open implies Japan closed):")
print(candidate_strategy_2)
print("\nCandidate Strategy 3 (manually fixed: Brazil/Germany/Mexico/U.S. open; India/Japan closed):")
print(candidate_strategy_3)

#-----------------------------------------------------------
# 4. Evaluate the performance of the candidate strategies.
#
# For evaluation, we will:
#    - Draw 100 samples from the exchange rate distribution (using the multivariate normal
#      with the Q1 mean vector and covariance matrix).
#    - For each candidate strategy, compute a cost using a cost function that uses the fixed
#      network (i.e. the candidate's binary decisions) and the drawn exchange rate sample.
#
# NOTE: In your actual application, you would “fix” the plant/line decisions in your optimization model.
# Here we demonstrate using a dummy cost function.
#-----------------------------------------------------------

# Q1 multivariate normal parameters (for [BRL, EUR, INR, JPY, MXN]):
mean_vector = np.array([4.389619, 0.895970, 72.852125, 120.345750, 19.028722])
cov_matrix = np.array([
    [0.866835, 0.007135, 5.424984, 6.776487, 0.756875],
    [0.007135, 0.001784, 0.129991, 0.427207, -0.007570],
    [5.424984, 0.129991, 47.403537, 83.177478, 2.410975],
    [6.776487, 0.427207, 83.177478, 254.423705, -8.938660],
    [0.756875, -0.007570, 2.410975, -8.938660, 2.970507]
])
rv = multivariate_normal(mean=mean_vector, cov=cov_matrix, seed=42)
num_samples = 100

# Define a dummy cost function that "evaluates" a fixed network strategy given an exchange rate sample.
# In practice, you would plug the fixed binary decisions into your cost model.
def compute_cost(strategy, exchange_sample):
    """
    Computes a dummy cost for a fixed network strategy.
    strategy: a pandas Series with entries for each plant decision, e.g., 'Brazil_Plant'
    exchange_sample: a numpy array for exchange rates [BRL, EUR, INR, JPY, MXN]
    """
    # For demonstration, assume each plant decision influences cost.
    # Let’s assume the cost for a plant is 100 if open, 150 if closed.
    plants = ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S._Plant']
    # Extract only the "Plant" decisions (we assume the columns for plants are named e.g., 'Brazil_Plant')
    plant_decisions = np.array([strategy.get(f"{plant}" if plant.endswith("_Plant") else f"{plant}_Plant", 0) for plant in ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S._Plant']])
    # Base cost: open = 100, closed = 150
    base_costs = np.where(plant_decisions == 1, 100, 150)
    cost = base_costs.sum()
    # Add an exchange-rate adjustment: for illustration, add cost * (mean(exchange_sample)/100)
    adjustment = cost * (np.mean(exchange_sample) / 100)
    return cost + adjustment

# Evaluate each candidate strategy over 100 exchange rate samples.
results = {}
for i, candidate in enumerate([candidate_strategy_1, candidate_strategy_2, candidate_strategy_3], start=1):
    cost_samples = []
    for _ in range(num_samples):
        sample = rv.rvs()
        cost_val = compute_cost(candidate, sample)
        cost_samples.append(cost_val)
    results[f"Strategy_{i}"] = {
        "Expected Cost": np.mean(cost_samples),
        "Cost Std": np.std(cost_samples)
    }

# Display evaluation results:
print("\nEvaluation of Candidate Strategies (over 100 exchange rate samples):")
for strat, res in results.items():
    print(f"{strat}: Expected Cost = {res['Expected Cost']:.2f}, Cost Std = {res['Cost Std']:.2f}")

#-----------------------------------------------------------
# 5. Recommend a strategy based on lowest expected cost (and possibly lower cost variability).
#-----------------------------------------------------------
recommended_strategy = min(results.items(), key=lambda x: x[1]["Expected Cost"])
print(f"\nRecommended Strategy: {recommended_strategy[0]}")

Q2 Results (first 5 rows):
   sample           optimal_configuration   total_cost
0       0  [1.0, 1.0, 0.0, 0.0, 1.0, 1.0]  1585.910068
1       1  [1.0, 1.0, 0.0, 0.0, 1.0, 1.0]  1396.808827
2       2  [1.0, 1.0, 0.0, 0.0, 1.0, 1.0]  2002.840311
3       3  [1.0, 1.0, 0.0, 0.0, 1.0, 1.0]  1970.685460
4       4  [1.0, 1.0, 0.0, 0.0, 1.0, 1.0]  2352.321112

Average decision values from Q2:
sample          49.500000
total_cost    1728.322667
dtype: float64

Candidate Strategy 1 (based on thresholding):
sample        1
total_cost    1
dtype: int64

Candidate Strategy 2 (enforcing Germany open implies Japan closed):
sample        1
total_cost    1
dtype: int64

Candidate Strategy 3 (manually fixed: Brazil/Germany/Mexico/U.S. open; India/Japan closed):
sample           1
total_cost       1
Brazil_Plant     1
Germany_Plant    1
Mexico_Plant     1
U.S._Plant       1
India_Plant      0
Japan_Plant      0
dtype: int64

Evaluation of Candidate Strategies (over 100 exchange rate samples):
Strategy